In [1]:
# [Cell 1] 기본 라이브러리 및 설정
import os
import json
import re
import torch
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer
from threading import Thread

# 설정값
INPUT_FILE = "project_full_context.txt"
OUTPUT_JSON = "project_flows.json"

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 실행 환경: {device.upper()}")

c:\Pyg\Projects\semi\yuzyproject-aimodels\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 실행 환경: CUDA


In [2]:
# [Cell 2] Qwen 2.5-Coder-1.5B 모델 로딩

# 모델 ID 변경 (Qwen 2.5 Coder 1.5B)
MODEL_ID = "Qwen/Qwen2.5-Coder-1.5B-Instruct"

# GPU 확인
print(f"🚀 실행 환경: {'CUDA (GPU)' if torch.cuda.is_available() else 'CPU'}")

try:
    print(f"🔄 모델 로딩 중... ({MODEL_ID})")
    
    # 1. 토크나이저 로딩
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
    
    # 2. 모델 로딩
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto",
        torch_dtype=torch.float16, # GPU 메모리 절약
        trust_remote_code=True
    )
    
    # Qwen 모델은 pad_token이 없을 수 있어 강제로 설정해줘야 에러가 안 난다.
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        
    print("✅ 모델 로딩 완료!")

except Exception as e:
    print(f"❌ 모델 로딩 실패: {e}")
    print("💡 힌트: CUDA 에러가 났다면 반드시 'Kernel Restart'를 먼저 해야 합니다.")

🚀 실행 환경: CUDA (GPU)
🔄 모델 로딩 중... (Qwen/Qwen2.5-Coder-1.5B-Instruct)


`torch_dtype` is deprecated! Use `dtype` instead!
Loading weights: 100%|██████████| 338/338 [00:00<00:00, 552.57it/s, Materializing param=model.norm.weight]                              


✅ 모델 로딩 완료!


In [3]:
# [Cell 3] 프롬프트 매니저 (React 프론트엔드 분석용)

SYSTEM_PROMPT = """
You are a 'Senior Frontend Developer' specializing in React.js.
Your task is to analyze the source code and generate a **Component & Data Flow Graph** in JSON.

### 🚨 CRITICAL ANALYSIS RULES
1. **Analyze Structure**: Identify `Pages` (Routes) and reusable `Components`.
2. **Trace API Calls**: Look for `fetch()` or `axios` calls inside `useEffect` or event handlers.
   - Example: `fetch('/api/tasks')` inside `TaskList.js`.
3. **Ignore Styles**: Do NOT analyze CSS/Style details. Focus on logic and structure.
4. **State Management**: Briefly note major states (`useState`, `useContext`).

### JSON OUTPUT FORMAT (Strict Structure)
{
  "frontend": [
    {
      "page": "Login",
      "file": "src/pages/Login.js",
      "description": "Login Page",
      "components": [],
      "apiCalls": [
        { "method": "POST", "url": "/api/auth/login", "trigger": "handleSubmit" }
      ]
    },
    {
      "page": "TaskList",
      "file": "src/pages/TaskList.js",
      "description": "View all tasks",
      "components": ["Navbar", "TaskItem"],
      "apiCalls": [
        { "method": "GET", "url": "/api/tasks", "trigger": "useEffect" }
      ]
    }
  ]
}
"""

print("✅ React 분석용 프롬프트 설정 완료")

✅ React 분석용 프롬프트 설정 완료


In [4]:
# [Cell 4] 분석 실행 (React 프론트엔드 모드 + 실시간 스트리밍)

def extract_json(text):
    text = re.sub(r"^```(json)?", "", text.strip(), flags=re.MULTILINE)
    text = re.sub(r"```$", "", text.strip(), flags=re.MULTILINE)
    start = text.find('{')
    end = text.rfind('}')
    if start == -1 or end == -1: return "{}"
    return text[start:end+1]

def run_full_scan():
    # 1. 메모리 청소
    gc.collect()
    torch.cuda.empty_cache()

    if not os.path.exists(INPUT_FILE):
        print(f"❌ 파일 없음: {INPUT_FILE}")
        return

    print(f"📂 파일 읽는 중... '{INPUT_FILE}'")
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        full_text = f.read()

    # =========================================================
    # 🧹 [스마트 필터링] 프론트엔드 코드만 남기기
    # =========================================================
    print("🧹 데이터 필터링 중... (Backend 제외, Frontend 유지)")
    
    filtered_lines = []
    is_skipping = False
    
    for line in full_text.split('\n'):
        if "### FILE:" in line:
            if "backend" in line or "package.json" in line or ".css" in line:
                is_skipping = True
            else:
                is_skipping = False
                filtered_lines.append(line)
        elif not is_skipping:
            filtered_lines.append(line)
            
    code_context = "\n".join(filtered_lines)
    print(f"📉 용량 압축 완료: {len(code_context)}자 (분석 준비 완료)")

    # 안전 자르기
    SAFE_LENGTH = 35000
    if len(code_context) > SAFE_LENGTH:
        code_context = code_context[:SAFE_LENGTH]
    
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Analyze the React source code. Visualize Pages, Components, and API calls:\n\n{code_context}"}
    ]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # 핵심: 스트리머 설정 (실시간 출력기)
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, decode_kwargs={"skip_special_tokens": True})

    # 생성 옵션 설정
    generation_kwargs = dict(
        **model_inputs,
        streamer=streamer,
        max_new_tokens=8000,
        do_sample=False,
        num_beams=1,
        repetition_penalty=1.1,
        early_stopping=True
    )

    # 별도 쓰레드에서 생성 시작 (그래야 화면이 안 멈춤)
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    print("\n✨ [Cursor AI Mode] Qwen 1.5B가 코드를 작성합니다...\n")
    print("-" * 60)

    # 실시간 출력 루프
    full_response = ""
    for new_text in streamer:
        print(new_text, end="", flush=True) # 타자기 효과
        full_response += new_text
    
    print("\n" + "-" * 60)
    print("✅ 생성 완료!")

    # JSON 파싱 및 저장
    json_str = extract_json(full_response)
    try:
        data = json.loads(json_str)
        output_filename = "project_flows_frontend.json"
        
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"📁 파일 저장됨: {output_filename}")
        return data

    except json.JSONDecodeError:
        print("❌ JSON 파싱 실패 (중간에 끊겼거나 형식이 잘못됨)")
        return None
    except Exception as e:
        print(f"❌ 에러 발생: {e}")
        return None

# 실행
json_data = run_full_scan()

📂 파일 읽는 중... 'project_full_context.txt'
🧹 데이터 필터링 중... (Backend 제외, Frontend 유지)
📉 용량 압축 완료: 36863자 (분석 준비 완료)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



✨ [Cursor AI Mode] Qwen 1.5B가 코드를 작성합니다...

------------------------------------------------------------
```json
{
  "frontend": [
    {
      "page": "Login",
      "file": "src/pages/Login.js",
      "description": "Login Page",
      "components": [],
      "apiCalls": [
        { "method": "POST", "url": "/api/auth/login", "trigger": "handleSubmit" }
      ]
    },
    {
      "page": "Register",
      "file": "src/pages/Register.js",
      "description": "Register Page",
      "components": [],
      "apiCalls": [
        { "method": "POST", "url": "/api/auth/register", "trigger": "handleSubmit" }
      ]
    },
    {
      "page": "TaskList",
      "file": "src/pages/TaskList.js",
      "description": "View all tasks",
      "components": ["Navbar", "TaskItem"],
      "apiCalls": [
        { "method": "GET", "url": "/api/tasks", "trigger": "useEffect" }
      ]
    },
    {
      "page": "TaskDetail",
      "file": "src/pages/TaskDetail.js",
      "description": "View a specific

In [5]:
# [Cell 5] 텍스트 트리 뷰어 (React 프론트엔드 구조 확인용)

# 방금 만든 파일명으로 설정
TARGET_FILE = "project_flows_frontend.json"

def print_frontend_tree(node, prefix="", is_last=True):
    """프론트엔드 JSON 구조에 맞춰 트리 출력"""
    connector = "└── " if is_last else "├── "
    
    # 1. 노드 이름 결정 (페이지 vs API vs 컴포넌트)
    if "page" in node:
        name = f"📄 Page: {node['page']} ({node.get('description', '')})"
        # 자식들을 임의로 합침 (컴포넌트 + API 호출)
        children = []
        
        # 컴포넌트 추가
        if "components" in node:
            for comp in node["components"]:
                children.append({"type": "component", "name": comp})
        
        # API 호출 추가
        if "apiCalls" in node:
            for api in node["apiCalls"]:
                api["type"] = "api"
                children.append(api)
                
    elif node.get("type") == "component":
        name = f"🧩 Comp: {node['name']}"
        children = []
        
    elif node.get("type") == "api":
        # API는 메서드, URL, 트리거 정보 표시
        trigger = f" (Trigger: {node.get('trigger', 'unknown')})" if "trigger" in node else ""
        name = f"📡 API: [{node.get('method')}] {node.get('url')}{trigger}"
        children = []
        
    else:
        name = "Unknown Node"
        children = []

    print(prefix + connector + name)
    
    # 2. 자식 노드 순회
    count = len(children)
    for i, child in enumerate(children):
        new_prefix = prefix + ("    " if is_last else "│   ")
        print_frontend_tree(child, new_prefix, i == count - 1)

def visualize_frontend_structure():
    if not os.path.exists(TARGET_FILE):
        print(f"❌ '{TARGET_FILE}' 파일이 없습니다. [Cell 4]를 먼저 실행하세요.")
        return

    with open(TARGET_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print(f"\n⚛️ React Frontend Flow Analysis ({TARGET_FILE})\n" + "="*50)
    
    if "frontend" in data:
        for page in data["frontend"]:
            print_frontend_tree(page)
            print("│")
    else:
        print("⚠️ 'frontend' 키를 찾을 수 없습니다. JSON 형식을 확인하세요.")

# 실행
visualize_frontend_structure()


⚛️ React Frontend Flow Analysis (project_flows_frontend.json)
└── 📄 Page: Login (Login Page)
    └── 📡 API: [POST] /api/auth/login (Trigger: handleSubmit)
│
└── 📄 Page: Register (Register Page)
    └── 📡 API: [POST] /api/auth/register (Trigger: handleSubmit)
│
└── 📄 Page: TaskList (View all tasks)
    ├── 🧩 Comp: Navbar
    ├── 🧩 Comp: TaskItem
    └── 📡 API: [GET] /api/tasks (Trigger: useEffect)
│
└── 📄 Page: TaskDetail (View a specific task)
    ├── 🧩 Comp: TaskHeader
    ├── 🧩 Comp: TaskDescription
    ├── 🧩 Comp: TaskAssignee
    ├── 🧩 Comp: TaskDueDate
    ├── 🧩 Comp: TaskStatus
    ├── 🧩 Comp: TaskPriority
    ├── 🧩 Comp: TaskComments
    ├── 📡 API: [GET] /api/tasks/:id (Trigger: useEffect)
    ├── 📡 API: [POST] /api/comments (Trigger: handleAddComment)
    └── 📡 API: [DELETE] /api/comments/:id (Trigger: handleDeleteComment)
│
└── 📄 Page: TaskForm (Create or edit a task)
    ├── 🧩 Comp: TaskTitle
    ├── 🧩 Comp: TaskDescription
    ├── 🧩 Comp: TaskStatus
    ├── 🧩 Comp: TaskPrio